### **6.4 - Making different paths in your DAGs with Branching**

Desde el principio del curso, hemos visto un DAG como una serie de tareas enlazadas donde esas tareas deben ser completadas para terminar el DagRun.  Bueno, Airflow trae una característica muy agradable para elegir entre diferentes tareas de acuerdo a un valor.  Ilustremos esto con un caso de uso.  

<center><img src="https://i.postimg.cc/gjfG8Kp3/a945.png"></center>

Digamos que estás trabajando en una empresa y necesitas localizar direcciones ip usando una API de geolocalización IP.  Como sabrás, la mayoría de las API tienen umbrales de petición por encima de los cuales ya no se pueden hacer peticiones.  Una forma de utilizar branching en Airflow sería crear una tarea con una lista de APIs en la que se comprueba cada API para saber si aún puede realizar peticiones. Dependiendo del resultado, se ejecutará la siguiente tarea correspondiente a la API disponible.  A continuación, el resultado se almacenará en una base de datos.  ¿Qué es exactamente el Branching? 

<center><img src="https://i.postimg.cc/8k3p2Tvs/a946.png"></center>

Branching es la forma de ir por un camino determinado en su DAG basado en una condición arbitraria que normalmente está relacionada con algo que sucedió en una tarea upstream. Esto se hace utilizando el BranchPythonOperator que devuelve el task_id de la tarea a ejecutar a continuación.  
Por ejemplo, tienes el siguiente DAG con tres tareas A, B y C dependiendo de la tarea implementada con el BranchPythonOperator aquí.  

<center><img src="https://i.postimg.cc/yYZ1B2hj/a947.png"></center>

Digamos que la tarea devuelve el id de tarea "task_c", por lo tanto, sólo se ejecutará "task_c" mientras que "task_a" y "task_b" se omitirán.  

<center><img src="https://i.postimg.cc/BnfqvNM0/a948.png"></center>

Muy bien, me gustaría decirte algo sobre el parámetro depends_on_past con branching. Si recuerdas, depends_on_past permite evitar que se ejecute una tarea en el DAGRun actual que no tuvo éxito en el DAGRun anterior. En la documentación de Airflow se indica que debido a que las tareas saltadas por el branch operator serán consideradas como fallidas, y por tanto no se lanzarán en el siguiente DAGRun si estableces las tareas downstream con el parámetro depends_on_past a True.  Bueno, he hecho diferentes experimentos basados en esta afirmación y, de hecho, parece que funciona.  Déjame mostrarte un ejemplo.  

<center><img src="https://i.postimg.cc/DzBFBhFv/a949.png"></center>

Supongamos que todas las tareas "tarea_a", "tarea_b" y "tarea_c" tienen el parámetro depends_on_past establecido en True.  Ahora, en el primer DAGRun, la tarea branch devuelve el id de tarea "task_c".  Por lo tanto, sólo se ejecuta la tarea_c, mientras que las tareas "task_b" y "task_a" se omiten. A continuación, en el segundo DAGRun, esta vez la tarea branch devuelve "task_b".  Pues bien, según la documentación, como esta tarea no se activó en el DAGRun anterior y tiene depends_on_past establecido en true, el scheduler no la ejecutará. De nuevo, basándome en mis experimentos, esto no es cierto.  La tarea task_b se activará como debería.  Por último, tenga cuidado cuando desee omitir una tarea.  

De hecho, si tomamos el siguiente DAG y la tarea branch no devuelve el id de tarea "task_a", entonces la última tarea "task_end" se saltará, lo cual no es algo que quieras. Como mejor práctica no deberías tener una ruta vacía (empty path) cuando utilices el BranchPythonOperator.  Si quieres saltarte una ruta, simplemente añade una tarea ficticia (dummy task) como se muestra aquí para que la última tarea "task_end" pueda ser ejecutada.  Muy bien, eso es todo para la teoría, pasemos a la práctica.  

<center><img src="https://i.postimg.cc/jS6RnNFJ/a950.png"></center>